# Get some data in

## Set libraries

In [21]:
import numpy as np
import xarray as xr
import rasterio
%matplotlib inline
from matplotlib.pyplot import *
from glob import glob
import os
import datetime
import pandas as pd
from rasterio_to_xarray import rasterio_to_xarray, xarray_to_rasterio

## Function to read in the MAIAC files into a data array

In [22]:
def maiac_file_to_da(filename):
    da = rasterio_to_xarray(filename)
    
    time_str = os.path.basename(filename)[17:-13]
    time_obj = datetime.datetime.strptime(time_str, '%Y%j%H%M')
    da.coords['time'] = time_obj
    
    return da

In [23]:
files = glob(r'D:\MAIAC_Data\Europe\h00v01\Projected\2005\*_proj.tif')

## Map runs function on every element of a list... creating a list of 2D arrays (the date)

In [24]:
list_of_das = map(maiac_file_to_da, files)

## Make the dataset 3D over the dimension time

In [25]:
MAIAC_AOT = xr.concat(list_of_das, 'time')

In [17]:
MAIAC_AOT.attrs.clear()

## Reorder time- because the A and T mess the time variable up and resampling wont work otherwise

In [26]:
reordered_MAIAC_AOT = MAIAC_AOT.isel(time=np.argsort(MAIAC_AOT.time))

## Put into DASK

In [19]:
ds = reordered_MAIAC_AOT.to_dataset(name='data')
ds.to_netcdf(r'D:\Annies_Dissertation\Methodology\Validation\AERONET\Test\test_year.nc')
all_data = xr.open_mfdataset(r'D:\Annies_Dissertation\Methodology\Validation\AERONET\Test\*.nc', chunks={'time':10})
alldata = all_data['data']

RuntimeError: Unknown error

In [ ]:
all_data.chunks

In [46]:
Daily_MAIAC_AOT = MAIAC_AOT.resample('D', dim='time', how='max')

In [ ]:
Daily_MAIAC_AOT

# Get AOT values for all of the years for Chilbolton pixel from MAIAC data

- find location of chilbolton pixel
 - get OS co-ordinates from https://www.ordnancesurvey.co.uk/gps/transformation
 - convert Lat and Long into Easting and Northing
 - Put Easting and Northing in and use attributes to convert into projection that the images are in to find cell row and column
 - extract data for that pixel

In [27]:
a = reordered_MAIAC_AOT.attrs['affine']

In [28]:
~a * (439480.737, 138506.177)

TypeError: bad operand type for unary ~: 'tuple'

In [50]:
Measurements = Daily_MAIAC_AOT.isel(x=1103, y=1027).dropna('time').to_dataframe(name='MAIAC_AOT')

In [51]:
Measurements.to_csv(r'D:\Annies_Dissertation\Methodology\Validation\Comparision\MAIAC\ALL_MAIAC_2005.csv')

- loop through all of the years data and save as csv of the AOT measurements (or appending if better)

# Read in AERONET data file into pandas DataFrame

In [1]:
import pandas as pd

In [2]:
def read_aeronet(filename):
    """Read a given AERONET AOT data file, and return it as a dataframe.
    
    This returns a DataFrame containing the AERONET data, with the index
    set to the timestamp of the AERONET observations. Rows or columns
    consisting entirely of missing data are removed. All other columns
    are left as-is.
    """
    dateparse = lambda x: pd.datetime.strptime(x, "%d:%m:%Y %H:%M:%S")
    aeronet = pd.read_csv(filename, skiprows=4, na_values=['N/A'],
                          parse_dates={'times':[0,1]},
                          date_parser=dateparse)

    aeronet = aeronet.set_index('times')
    del aeronet['Julian_Day']
    
    # Drop any rows that are all NaN and any cols that are all NaN
    # & then sort by the index
    an = (aeronet.dropna(axis=1, how='all')
                .dropna(axis=0, how='all')
                .rename(columns={'Last_Processing_Date(dd/mm/yyyy)': 'Last_Processing_Date'})
                .sort_index())

    return an

In [3]:
Aeronet = read_aeronet(r'D:\Annies_Dissertation\Methodology\Validation\Comparision\050101_140630_Chilbolton.lev20')

In [4]:
Aeronet[:10]

,AOT_1640,AOT_1020,AOT_870,AOT_675,AOT_500,AOT_440,AOT_380,AOT_340,Water(cm),440-870Angstrom,...,N[AOT_500],N[AOT_440],N[AOT_380],N[AOT_340],N[Water(cm)],N[440-870Angstrom],N[380-500Angstrom],N[440-675Angstrom],N[500-870Angstrom],N[340-440Angstrom]
times,,,,,,,,,,,,,,,,,,,,,
2005-10-10,NaN,0.086215,0.101031,0.158985,0.273962,0.334494,0.414908,0.478670,1.985016,1.776144,...,13,13,13,13,13,13,13,13,13,13
2005-10-11,NaN,0.181416,0.190722,0.239113,0.332206,0.377564,0.434911,0.479563,2.838769,1.015444,...,4,4,4,4,4,4,4,4,4,4
2005-10-15,NaN,0.089029,0.106889,0.160082,0.235446,0.266397,0.297522,0.317088,1.122777,1.403509,...,21,21,21,21,21,21,21,21,21,21
2005-10-16,NaN,0.104994,0.118630,0.173252,0.248879,0.278775,0.310195,0.331838,1.657206,1.260693,...,6,6,6,6,6,6,6,6,6,6
2005-10-17,NaN,0.079702,0.095950,0.151347,0.258691,0.313295,0.383490,0.435075,1.867112,1.762484,...,7,7,7,7,7,7,7,7,7,7
2005-10-25,NaN,0.098682,0.087635,0.091231,0.098083,0.099108,0.105640,0.113873,1.484080,0.192620,...,3,3,3,3,3,3,3,3,3,3
2005-10-27,NaN,0.047911,0.047767,0.065196,0.105974,0.126841,0.154790,0.176768,1.822174,1.456694,...,16,16,16,16,16,16,16,16,16,16
2005-10-31,NaN,0.070198,0.063919,0.067750,0.080819,0.086297,0.096940,0.107747,1.432144,0.460244,...,4,4,4,4,4,4,4,4,4,4
2005-11-01,NaN,0.041190,0.030583,0.035815,0.043239,0.043858,0.047812,0.053201,1.173914,0.569022,...,19,19,19,19,19,19,19,19,19,19
